In [4]:
import pandas as pd
import numpy as np

In [5]:
path_to_data = r"C:\Users\Rafay\Documents\thesis3\thesis\ActualWork\e2e\cache"
factors_list = ['RF']

# Data Load

In [6]:
def LoadData(path_to_data, e2e=True, datatype='broad'):
    if e2e:
        path_to_returns = r'{}\asset_weekly_{}.pkl'.format(path_to_data, datatype)
        path_to_prices = r'{}\assetprices_weekly_{}.pkl'.format(path_to_data, datatype)
        path_to_factors = r'{}\factor_weekly_{}.pkl'.format(path_to_data, datatype)

        returns = pd.read_pickle(path_to_returns)
        prices = pd.read_pickle(path_to_prices)
        factors = pd.read_pickle(path_to_factors)

        assets_list = prices.columns.to_list()

        returns = returns.reset_index()
        prices = prices.reset_index()
        factors = factors.reset_index()

        factors = factors.rename(columns={"Date": "date", "Mkt-RF": "RF"})
        factors = factors[['date'] + factors_list]

        return returns, assets_list, prices, factors

    path_to_prices = r'{}\prices.csv'.format(path_to_data)
    path_to_factors = r'{}\3factors.csv'.format(path_to_data)

    prices = pd.read_csv(path_to_prices)
    factors = pd.read_csv(path_to_factors)

    assets_list = list(prices['symbol'].unique())

    assets_list_cleaned = [x for x in assets_list if str(x) != 'nan']
    pivot_prices = np.round(pd.pivot_table(prices, values='close', 
                                    index='date', 
                                    columns='symbol', 
                                    aggfunc=np.mean),2)
    pivot_prices = pivot_prices.reset_index()
    pivot_prices['date'] = pd.to_datetime(pivot_prices['date'])
    factors['date'] = pd.to_datetime(factors['Date'], format="%Y%m%d")

    pivot_prices = pivot_prices.set_index('date')
    returns = pivot_prices.pct_change()
    pivot_prices = pivot_prices.reset_index()
    returns = returns.reset_index()
    returns = returns.merge(factors, on='date', how='left')
    returns = returns.drop(['Date'], axis=1)
    returns = returns.dropna()

    return returns, assets_list_cleaned, pivot_prices, []


In [7]:
datatype='cross_asset'
returns, assets_list_cleaned, prices, factors = LoadData(path_to_data, e2e=True, datatype=datatype)

In [102]:
returns.head()

,date,FTGC,BNDX,SPY
0,2014-01-03,-0.008560,-0.001609,0.003071
1,2014-01-10,0.003557,-0.004206,0.003688
2,2014-01-17,-0.000838,-0.002796,-0.013519
3,2014-01-24,-0.019560,-0.001197,0.034661
4,2014-01-31,-0.032443,-0.005337,0.022047


In [8]:
import torch

,date
287,2019-07-05
288,2019-07-12
289,2019-07-19
290,2019-07-26
291,2019-08-02
...,...
386,2021-05-28
387,2021-06-04
388,2021-06-11
389,2021-06-18


In [107]:
lookback = 104
date = '2021-07-02'
returns_lastn = returns[(returns['date'] < date)].tail(lookback)
asset_returns = returns_lastn.drop(['FTGC', 'BNDX', 'SPY'], axis=1)

In [108]:
asset_returns

,date
287,2019-07-05
288,2019-07-12
289,2019-07-19
290,2019-07-26
291,2019-08-02
...,...
386,2021-05-28
387,2021-06-04
388,2021-06-11
389,2021-06-18


287    1562284800000000000
288    1562889600000000000
289    1563494400000000000
290    1564099200000000000
291    1564704000000000000
              ...         
386    1622160000000000000
387    1622764800000000000
388    1623369600000000000
389    1623974400000000000
390    1624579200000000000
Name: date, Length: 104, dtype: int64

In [116]:
asset_returns_tensor.size(0) - window_size + 1

53

In [1]:
from enum import Enum
import numpy as np
import cvxpy as cp
import math
from util import nearestPD, BatchUtils

import numpy as np
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable

import PortfolioClasses as pc
import LossFunctions as lf

from torch.utils.data import DataLoader

In [124]:
def drrpw_nominal_learnDelta_batched(n_y):
    # Variables
    phi = cp.Variable((n_y,1), nonneg=True)
    t = cp.Variable()
    
    # Size of uncertainty set
    delta = cp.Parameter(1, nonneg=True)

    # L - Square Root of Covariance Matrix
    L = cp.Parameter((n_y, n_y))

    # Norm for x
    p = 2

    # Kappa, dont need this to be trainable as the value of this doesnt really matter
    k = 100

    # Constraints
    constraints = [
        phi >= 0,
        t >= cp.power(cp.norm(L@phi, 2) + delta*cp.norm(phi, p),2)
        # t >= cp.power(cp.norm(L@phi, 2) + cp.norm(T@phi, 2),2)
    ]

    log_term = 0
    for i in range(n_y):
        log_term += cp.log(phi[i])

    obj = (t) - k*log_term

    # Objective function
    objective = cp.Minimize(obj)    

    # Construct optimization problem and differentiable layer
    problem = cp.Problem(objective, constraints=constraints)

    return CvxpyLayer(problem, parameters=[delta, L], variables=[phi, t])

In [60]:
def DRRPW(mu,Q, delta=0):
    
    # # of Assets
    n = 3

    # Decision Variables
    w = cp.Variable(n)
    t = cp.Variable()
    
    # Kappa
    k = 100

    # Norm for x
    p = 2

    L = np.linalg.cholesky(Q)
    L /= np.linalg.norm(L)

    constraints = [
        w>=0, # Disallow Short Sales
        t >= cp.power(cp.norm(L@w, 2) + delta*cp.norm(w, p),2)
    ]

    # risk = cp.quad_form(w, Q)

    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    obj = t - k*log_term

    prob = cp.Problem(cp.Minimize(obj), constraints=constraints)
    
    # ECOS fails sometimes, if it does then do SCS
    try:
        prob.solve(verbose=False)
    except:
        prob.solve(solver='SCS',verbose=False)
    
    x = w.value
    x = np.divide(x, np.sum(x))

    return x

In [78]:
Q = np.cov(asset_returns.to_numpy(), rowvar=False)
layer = drrpw_nominal_learnDelta(Q)
delta = torch.FloatTensor(1).uniform_(0.001, 0.002)
print(delta)
z_star, _ = layer(delta)
print(z_star)
smax = torch.nn.Softmax(dim=1)
z_star = smax(z_star)
print(z_star)

tensor([0.0012])
tensor([[10.0254, 79.9962, 11.8627]])
tensor([[4.0934e-31, 1.0000e+00, 2.5704e-30]])


tensor([[0.0987, 0.7849, 0.1164]])

In [66]:
DRRPW(None, Q, delta=0.0016)

array([0.09873581, 0.78478117, 0.11648302])

In [11]:
def compute_covariance_matrix(sliding_windows):
    # get the number of batches and number of assets
    num_batches, window_size, num_assets = sliding_windows.size()

    # initialize covariance matrix tensor
    covariance_matrix_tensor = torch.zeros((num_batches, num_assets, num_assets))

    # compute covariance matrix for each batch
    for i in range(num_batches):
        batch = sliding_windows[i]
        covariance_matrix = torch.Tensor(np.cov(batch.numpy().T))
        covariance_matrix_tensor[i] = covariance_matrix

    return covariance_matrix_tensor


In [12]:
compute_covariance_matrix(sliding_windows)

: 

: 

In [5]:
%pip install ics


     ---------------------------------------- 40.1/40.1 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 101.5/101.5 kB 5.7 MB/s eta 0:00:00


In [23]:
import ics
import datetime

In [50]:
from ics import Calendar, Event

In [57]:
i = 0
c = Calendar()
for week in range(15):
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        date = date_list[i]
        # print(plan.loc[plan['WEEK'] == float(week)])
        do = plan.loc[plan['WEEK'] == (week+1)][day].values[0]
        i+=1

        e = Event(name="HM Training Day {}".format(i), begin=date, description=do)
        c.events.add(e)    

In [58]:
with open('my.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())

In [22]:
plan = plan.dropna(subset='WEEK')
plan.head(15)

,WEEK,Date,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Weekly Mileage,Unnamed: 10
0,1.0,NaN,Rest Day,Walk / Run 10 x \n2 min walk\n1 min run,Rest Day,Walk / Run 10 x \n2 min walk\n1 min run,Strength Training 45-60 mins,Rest Day,Long Run 2.5k (walk if needed),Around\n9k,5K PHASE
3,2.0,NaN,Rest Day,Walk / Run 15 x \n1 min walk\n1 min run,Rest Day,Walk / Run 15 x \n1 min walk\n1 min run,Strength Training 45-60 mins,Walk / Run 15 x \n1 min walk\n1 min run,Long Run 3k (walk if needed),Around\n13k,NaN
6,3.0,NaN,Rest Day,Walk / Run 15 x \n0.5 min walk\n1.5 min run,Rest Day,Walk / Run 15 x \n0.5 min walk\n1.5 min run,Strength Training 45-60 mins,Walk / Run 15 x \n0.5 min walk\n1.5 min run,Long Run 4k (walk if needed),Around\n15k,NaN
9,4.0,NaN,Rest Day,Walk / Run 10 x \n1 min walk\n2 min run,Rest Day,Walk / Run 10 x \n1 min walk\n2 min run,Strength Training 45-60 mins,Rest Day,5 k,Around\n13k,NaN
12,5.0,NaN,Rest Day,4 k,3 k,Rest Day,4 k,Strength Training,6 k,17 k,10K PHASE
15,6.0,NaN,Rest Day,4 k,3 k,Rest Day,4 k,Strength Training,7 k,18 k,NaN
18,7.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,8 k,25 k,NaN
21,8.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,10 k,27 k,NaN
24,9.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,8 k,25 k,HALF MARATHON PHASE
27,10.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,11 k,28 k,NaN


# (i) Ledoit-Wolfe Shrinkage

In [7]:
def shrinkage(returns):
    """Shrinks sample covariance matrix towards constant correlation unequal variance matrix.
    Ledoit & Wolf ("Honey, I shrunk the sample covariance matrix", Portfolio Management, 30(2004),
    110-119) optimal asymptotic shrinkage between 0 (sample covariance matrix) and 1 (constant
    sample average correlation unequal sample variance matrix).
    Paper:
    http://www.ledoit.net/honey.pdf
    Matlab code:
    https://www.econ.uzh.ch/dam/jcr:ffffffff-935a-b0d6-ffff-ffffde5e2d4e/covCor.m.zip
    Special thanks to Evgeny Pogrebnyak https://github.com/epogrebnyak
    :param returns:
        t, n - returns of t observations of n shares.
    :return:
        Covariance matrix, sample average correlation, shrinkage.
    """
    t, n = returns.shape
    mean_returns = np.mean(returns, axis=0, keepdims=True)
    returns -= mean_returns
    sample_cov = returns.transpose() @ returns / t

    # sample average correlation
    var = np.diag(sample_cov).reshape(-1, 1)
    sqrt_var = var ** 0.5
    unit_cor_var = sqrt_var * sqrt_var.transpose()
    average_cor = ((sample_cov / unit_cor_var).sum() - n) / n / (n - 1)
    prior = average_cor * unit_cor_var
    np.fill_diagonal(prior, var)

    # pi-hat
    y = returns ** 2
    phi_mat = (y.transpose() @ y) / t - sample_cov ** 2
    phi = phi_mat.sum()

    # rho-hat
    theta_mat = ((returns ** 3).transpose() @ returns) / t - var * sample_cov
    np.fill_diagonal(theta_mat, 0)
    rho = (
        np.diag(phi_mat).sum()
        + average_cor * (1 / sqrt_var @ sqrt_var.transpose() * theta_mat).sum()
    )

    # gamma-hat
    gamma = np.linalg.norm(sample_cov - prior, "fro") ** 2

    # shrinkage constant
    kappa = (phi - rho) / gamma
    shrink = max(0, min(1, kappa / t))

    # estimator
    sigma = shrink * prior + (1 - shrink) * sample_cov

    return sigma, average_cor, shrink

In [8]:
sigma, average_cor, shrink = shrinkage(returns.drop('date', axis=1).to_numpy())

In [9]:
sigma

array([[4.11719612e-04, 1.05466722e-05, 1.72681015e-04],
       [1.05466722e-05, 2.97840162e-05, 3.20162546e-05],
       [1.72681015e-04, 3.20162546e-05, 5.96583123e-04]])